In [1]:
import torch
from mmengine import dump, load
from clip import clip

/home/wuhao/.conda/envs/torch2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
label_info = load('../../configs/label_info.json')
hoi_info = label_info['hoi_info']
ins_info = label_info['ins_info']

In [6]:
hoi_text = [item['text'] for item in sorted(list(hoi_info.values()), key=lambda x:x['valid_id'])]
hoi_text_inputs = torch.cat([clip.tokenize(text) for text in hoi_text])
ins_text = [item['text'] for item in sorted(list(ins_info.values()), key=lambda x:x['valid_id'])]
ins_text.append('a photo of nothing')
ins_text_inputs = torch.cat([clip.tokenize(text) for text in ins_text])

In [4]:
def gen_text_embed(text, clip_model):
    with torch.no_grad():
        # text_embed = clip_model.encode_text(text.to('cuda')).float()
        text_embed = clip_model.encode_text(text).float()
    return text_embed / text_embed.norm(dim=-1, keepdim=True)

In [5]:
# clip_model, _ = clip.load(name='ViT-B/32', download_root='/data1/wuhao/weights/CLIP', device='cpu')
# clip_model, _ = clip.load(name='/data1/wuhao/weights/CLIP/RN50.pt', device='cpu')
clip_model, _ = clip.load(name='/data1/wuhao/weights/CLIP/ViT-B-32.pt', device='cpu')
# clip_model, _ = clip.load(name='/data1/wuhao/weights/CLIP/ViT-L-14.pt', device='cpu')

In [7]:
hoi_text_embed = gen_text_embed(hoi_text_inputs, clip_model).cpu()
ins_text_embed = gen_text_embed(ins_text_inputs, clip_model).cpu()

In [8]:
# dump(hoi_text_embed, 'hoi_text_embed_CLIP-1024_RN50.pkl')
# dump(ins_text_embed, 'ins_text_embed_CLIP-1024_RN50.pkl')
dump(hoi_text_embed, 'hoi_text_embed_CLIP-512_ViT-B-32.pkl')
dump(ins_text_embed, 'ins_text_embed_CLIP-512_ViT-B-32.pkl')
# dump(hoi_text_embed, 'hoi_text_embed_CLIP-768_ViT-L-14.pkl')
# dump(ins_text_embed, 'ins_text_embed_CLIP-768_ViT-L-14.pkl')